### Алгоритмы интеллектуальной обработки больших объемов данных
## Домашнее задание №3 - Дерево решений


**Общая информация**

**Срок сдачи:** 25 ноября 2019, 15:00   
**Штраф за опоздание:** -2 балла после 15:00 25 ноября, -4 балла после 15:00 2 декабря, -6 баллов после 15:00 9 декабря  -8 баллов после 15:00 16 декабря.

При отправлении ДЗ указывайте фамилию в названии файла Присылать ДЗ необходимо в виде ссылки на свой github репозиторий на почту ml1.sphere@mail.ru с указанием темы в следующем формате:
[ML0919, Задание 3] Фамилия Имя. 


Используйте данный Ipython Notebook при оформлении домашнего задания.

###### Задание 1 (3 балла)
Разберитесь в коде MyDecisionTreeClassifier, который уже частично реализован. Допишите код там, где написано "Ваш код". Ваша реализация дерева должна работать по точности не хуже DecisionTreeClassifier из sklearn. Точность проверяется на [wine](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_wine.html) и [Speed Dating Data](https://cloud.mail.ru/public/8nHV/p6J7wY1y1)

###### Задание 2 (3 балла)
Добиться скорости работы на fit не медленнее чем в 10 раз sklearn на данных wine и Speed Dating Data. 
Для этого используем numpy.

###### Задание 3 (2 балла)
Добавьте функционал, который определяет значения feature importance. Выведите 10 главных фичей под пунктом Задание 4 (уже написано ниже) для MyDecisionTreeClassifier и DecisionTreeClassifier так, чтобы сразу были видны выводы и по MyDecisionTreeClassifier, и по DecisionTreeClassifier. Используем данные Speed Dating Data.

###### Задание 4 (2 балла)
С помощью GridSearchCV или RandomSearchCV подберите наиболее оптимальные параметры для случайного леса (Выберете 2-3 параметра). Используем данные Speed Dating Data. Задание реализуйте под пунктом Задание 5 (уже написано ниже)


**Штрафные баллы:**

1. Невыполнение PEP8 -1 балл
2. Отсутствие фамилии в имени скрипта (скрипт должен называться по аналогии со stroykova_hw3.ipynb) -1 балл
3. Все строчки должны быть выполнены. Нужно, чтобы output команды можно было увидеть уже в git'е. В противном случае -1 балл
4. При оформлении ДЗ нужно пользоваться данным файлом в качестве шаблона. Не нужно удалять и видоизменять написанный код и текст. В противном случае -1 балл

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.datasets import load_wine
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import KFold, train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier

%matplotlib inline
%load_ext pycodestyle_magic

In [706]:
#%%pycodestyle


class MyDecisionTreeClassifier:
    NON_LEAF_TYPE = 0
    LEAF_TYPE = 1

    def __init__(self, min_samples_split=2, max_depth=None, sufficient_share=1.0,
                 criterion='gini', max_features=None):
        self.tree = dict()
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.sufficient_share = sufficient_share
        self.num_class = -1
        self.feature_importances_ = None
        if criterion == 'gini':
            self.G_function = self.__gini
        elif criterion == 'entropy':
            self.G_function = self.__entropy
        elif criterion == 'misclass':
            self.G_function = self.__misclass
        else:
            print('invalid criterion name')
            raise

        if max_features == 'sqrt':
            self.get_feature_ids = self.__get_feature_ids_sqrt
        elif max_features == 'log2':
            self.get_feature_ids = self.__get_feature_ids_log2
        elif max_features == None:
            self.get_feature_ids = self.__get_feature_ids_N
        else:
            print('invalid max_features name')
            raise

    def __gini(self, l_c, l_s, r_c, r_s):
        l_s = l_s.astype('float')
        r_s = r_s.astype('float')
        return  -(np.sum(l_c*l_c, axis=1)/(l_s*(l_s+r_s)) +
                  np.sum(r_c*r_c, axis=1)/(r_s*(l_s+r_s)))
    
    def __entropy(self,  l_c, l_s, r_c, r_s):
        return -(np.sum(np.log(l_c.T/l_s + 0.00001).T*l_c, axis=1)/(l_s+r_s) +
                 np.sum(np.log(r_c.T/r_s + 0.00001).T*r_c, axis=1)/(l_s+r_s))

    def __misclass(self, l_c, l_s, r_c, r_s):
        return -(np.max(l_c.T/l_s, axis=0) + np.max(r_c.T/r_s, axis=0))

    def __get_feature_ids_sqrt(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        return feature_ids[ : n_feature**0.5]
        
    def __get_feature_ids_log2(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        return feature_ids[ : log2(n_feature)]

    def __get_feature_ids_N(self, n_feature):
        return range(n_feature)
    
    def __sort_samples(self, x, y):
        sorted_idx = x.argsort()
        return x[sorted_idx], y[sorted_idx]

    def __div_samples(self, x, y, feature_id, threshold):
        left_mask = x[:, feature_id] > threshold
        right_mask = ~left_mask
        return x[left_mask], x[right_mask], y[left_mask], y[right_mask]

    def __find_threshold(self, x, y):
        
        best_c = 3
        best_i = -1
        threshold = -1
        ind = self.get_feature_ids(x[0].size)
        for i in ind:
            unique_values, unique_mask = np.unique(x[:, i], return_index=True)
            mask = (y[unique_mask][:-1] ^ y[unique_mask][1:]).astype('bool')
            thresholds = (unique_values[np.insert(mask, 0, 0)] 
                          + unique_values[np.insert(mask, len(mask), 0)]) / 2
            split = [self.__div_samples(x, y, i, threshold)[2:4]
                       for threshold in thresholds]
            l_c = np.asarray([np.bincount(c[0], minlength=self.num_class) for c in split])
            r_c = np.asarray([np.bincount(c[1], minlength=self.num_class) for c in split])
            if(l_c.size == 0 or r_c.size == 0):
                continue
            l_s = np.asarray([c[0].size for c in split])
            r_s = np.asarray([c[1].size for c in split])

            cur_c = self.G_function(l_c, l_s, r_c, r_s)
            tmp_c = cur_c[np.argmin(cur_c)]
            if(tmp_c < best_c):
                best_c = tmp_c
                threshold = thresholds[np.argmin(cur_c)]
                best_i = i
        
        self.feature_importances_[best_i] += x[0].size
            
        return best_i, threshold

            
    def __fit_node(self, x, y, node_id, depth):
        if(depth == self.max_depth or y.size < self.min_samples_split
           or np.unique(y).size <= 1):
            count = np.bincount(y)
            self.tree[node_id] = (self.__class__.LEAF_TYPE,
                                  np.argmax(count), np.max(count)/y.size)
            return
        
        feature_id, threshold = self.__find_threshold(x, y)
        if(feature_id == -1):
            count = np.bincount(y)
            self.tree[node_id] = (self.__class__.LEAF_TYPE,
                                  np.argmax(count), np.max(count)/y.size)
            return
        x_l, x_r, y_l, y_r = self.__div_samples(x, y, feature_id, threshold)
        self.tree[node_id] = (self.__class__.NON_LEAF_TYPE, feature_id, threshold)
        self.__fit_node(x_l, y_l, node_id*2+1, depth+1)
        self.__fit_node(x_r, y_r, node_id*2+2, depth+1)
    
    def fit(self, x, y):
        self.num_class = np.unique(y).size
        self.feature_importances_ = np.zeros(x[0].size)
        self.__fit_node(x, y, 0, 0) 
        self.feature_importances_ /= x.size

    def __predict_class(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_class(x, 2 * node_id + 1)
            else:
                return self.__predict_class(x, 2 * node_id + 2)
        else:
            return node[1]

    def __predict_probs(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_probs(x, 2 * node_id + 1)
            else:
                return self.__predict_probs(x, 2 * node_id + 2)
        else:
            return node[2]
        
    def predict(self, X):
        return np.array([self.__predict_class(x, 0) for x in X])
    
    def predict_probs(self, X):
        return np.array([self.__predict_probs(x, 0) for x in X])

    def fit_predict(self, x_train, y_train, predicted_x):
        self.fit(x_train, y_train)
        return self.predict(predicted_x)


In [748]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2, criterion='gini')
clf = DecisionTreeClassifier(min_samples_split=2)

In [749]:
wine = load_wine()
X_train, X_test, y_train, y_test = train_test_split(wine.data, wine.target,
                                                    test_size=0.1, stratify=wine.target)

## Проверка скорости работы на wine

In [755]:
%time clf.fit(X_train, y_train)

CPU times: user 2.32 ms, sys: 81 µs, total: 2.4 ms
Wall time: 6.62 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [751]:
%time my_clf.fit(X_train, y_train)

CPU times: user 34.5 ms, sys: 2 µs, total: 34.5 ms
Wall time: 36.7 ms


## Проверка качества работы на wine

In [752]:
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

0.9487179487179486

In [753]:
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

0.9487179487179486

## Подготовка данных Speed Dating Data 

In [760]:
df = pd.read_csv('data/speed-dating-experiment/Speed Dating Data.csv'
                 , sep=',', encoding="ISO-8859-1")
df = df.iloc[:, :97]
df = df.drop(['id'], axis=1)
df = df.drop(['idg'], axis=1)
df.drop_duplicates(subset=['iid']).gender.value_counts()
df = df.drop(['condtn'], axis=1)
df = df.drop(['round'], axis=1)
df = df.drop(['position', 'positin1'], axis=1)
df = df.drop(['order'], axis=1)
df = df.drop(['partner'], axis=1)
df = df.drop(['age_o', 'race_o', 'pf_o_att','pf_o_sin',
              'pf_o_int','pf_o_fun', 'pf_o_amb', 'pf_o_sha','dec_o',
              'attr_o', 'sinc_o', 'intel_o', 'fun_o','amb_o', 'shar_o',
              'like_o', 'prob_o', 'met_o'],axis=1)
df = df.dropna(subset=['age'])


df.loc[:, 'tuition'] = df.loc[:, 'tuition'].str.replace(',', '').astype(np.float)
df.loc[:, 'tuition'] = df.tuition.fillna(-999)
df = df.dropna(subset=['imprelig', 'imprace'])
df = df.drop(['from', 'zipcode'], axis=1)
df.loc[:, 'income'] = df.loc[:, 'income'].str.replace(',', '').astype(np.float)
df.loc[:, 'income'] = df.loc[:, 'income'].fillna(-999)
df = df.dropna(subset=['date'])
df.loc[:, 'career_c'] = df.loc[:, 'career_c'].fillna(0)
df = df.drop(['career'], axis=1)
df = df.drop(['sports', 'tvsports', 'exercise','dining'
              , 'museums', 'art', 'hiking', 'gaming',
              'clubbing', 'reading', 'tv','theater', 'movies',
              'concerts','music', 'shopping', 'yoga'], axis=1)
df = df.drop(['expnum'], axis=1)
df.loc[:, 'field_cd'] = df.loc[:, 'field_cd'].fillna(0)
df = df.drop(['field'], axis=1)
pd.get_dummies(df, columns=['field_cd'],prefix='field_cd', prefix_sep='=')


df = df.drop(['undergra'], axis=1)
df.loc[:, 'mn_sat'] = df.loc[:, 'mn_sat'].str.replace(',', '').astype(np.float)
df.loc[:, 'mn_sat'] = df.mn_sat.fillna(-999)
df.loc[:, 'temp_totalsum'] = df.loc[:, ['attr1_1', 'sinc1_1',
                                        'intel1_1','fun1_1', 'amb1_1', 'shar1_1']].sum(axis=1)
df.loc[:, ['attr1_1', 'sinc1_1','intel1_1', 'fun1_1'
           , 'amb1_1', 'shar1_1']] = (df.loc[:, ['attr1_1', 'sinc1_1','intel1_1', 'fun1_1','amb1_1', 'shar1_1']].T / df.loc[:, 'temp_totalsum'].T).T * 100
df.loc[:, 'temp_totalsum'] = df.loc[:, ['attr2_1', 'sinc2_1','intel2_1',
                                        'fun2_1','amb2_1', 'shar2_1']].sum(axis=1)
df.loc[:, ['attr2_1', 'sinc2_1','intel2_1',
           'fun2_1','amb2_1', 'shar2_1']] = (df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1', 'amb2_1',
                                                        'shar2_1']].T / df.loc[:, 'temp_totalsum'].T).T * 100
df = df.drop(['temp_totalsum'], axis=1)


for i in [4, 5]:
    t_drop = ['attr{}_1'.format(i), 'sinc{}_1'.format(i),'intel{}_1'.format(i), 'fun{}_1'.format(i), 'amb{}_1'.format(i), 'shar{}_1'.format(i)]
    if i != 4:
        t_drop.remove('shar{}_1'.format(i))
    df = df.drop(t_drop, axis=1)
    
    
df = df.drop(['wave'], axis=1)
df_male = df.query('gender == 1').drop_duplicates(subset=['iid', 'pid']) .drop(['gender'], axis=1) .dropna()
df_female = df.query('gender == 0').drop_duplicates(subset=['iid']) .drop(['gender'
                                                                           , 'match', 'int_corr', 'samerace'], axis=1) .dropna()
df_female.columns = df_female.columns + '_f'
df_female = df_female.drop(['pid_f'], axis=1)
df_pair = df_male.join(df_female.set_index('iid_f'),on='pid',how='inner')
df_pair = df_pair.drop(['iid', 'pid'], axis=1)

In [761]:
df_pair

,match,int_corr,samerace,age,field_cd,mn_sat,tuition,race,imprace,imprelig,...,sinc2_1_f,intel2_1_f,fun2_1_f,amb2_1_f,shar2_1_f,attr3_1_f,sinc3_1_f,fun3_1_f,intel3_1_f,amb3_1_f
100,0,0.14,0,27.0,8.0,-999.0,-999.0,2.0,7.0,3.0,...,20.0,15.0,20.0,5.0,5.0,6.0,8.0,8.0,8.0,7.0
110,0,0.54,0,22.0,1.0,-999.0,-999.0,2.0,1.0,1.0,...,20.0,15.0,20.0,5.0,5.0,6.0,8.0,8.0,8.0,7.0
120,1,0.16,1,22.0,1.0,-999.0,-999.0,4.0,3.0,5.0,...,20.0,15.0,20.0,5.0,5.0,6.0,8.0,8.0,8.0,7.0
130,1,0.61,0,23.0,1.0,-999.0,-999.0,2.0,1.0,1.0,...,20.0,15.0,20.0,5.0,5.0,6.0,8.0,8.0,8.0,7.0
140,1,0.21,0,24.0,1.0,-999.0,-999.0,3.0,3.0,1.0,...,20.0,15.0,20.0,5.0,5.0,6.0,8.0,8.0,8.0,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8289,0,0.23,0,30.0,8.0,-999.0,-999.0,2.0,8.0,8.0,...,10.0,10.0,10.0,10.0,20.0,3.0,9.0,9.0,9.0,7.0
8311,0,0.29,0,28.0,8.0,-999.0,-999.0,2.0,2.0,3.0,...,10.0,10.0,10.0,10.0,20.0,3.0,9.0,9.0,9.0,7.0
8333,0,-0.20,0,30.0,8.0,-999.0,-999.0,2.0,5.0,6.0,...,10.0,10.0,10.0,10.0,20.0,3.0,9.0,9.0,9.0,7.0
8355,0,-0.32,0,27.0,8.0,1400.0,26019.0,1.0,2.0,1.0,...,10.0,10.0,10.0,10.0,20.0,3.0,9.0,9.0,9.0,7.0


In [762]:
X = df_pair.iloc[:, 1:].values
y = df_pair.iloc[:, 0].values

In [768]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)
my_clf = MyDecisionTreeClassifier()
clf = DecisionTreeClassifier()

## Проверка скорости работы на Speed Dating Data 

In [769]:
%time clf.fit(X_train,y_train)

CPU times: user 71.4 ms, sys: 0 ns, total: 71.4 ms
Wall time: 71.7 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [770]:
%time my_clf.fit(X_train, y_train)

/home/scrinium/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:38: RuntimeWarning: invalid value encountered in true_divide


CPU times: user 4.92 s, sys: 0 ns, total: 4.92 s
Wall time: 5.01 s


## Проверка качества работы на Speed Dating Data

In [771]:
f1_score(y_pred=clf.predict(X_test),y_true=y_test,average='macro')

0.5517508289143821

In [772]:
f1_score(y_pred=my_clf.predict(X_test),y_true=y_test,average='macro')

0.555217932203763

## Задание 4

In [773]:
df_pair.iloc[:, 1:].columns[np.argsort(-clf.feature_importances_)[:11]]

Index(['int_corr', 'income_f', 'intel2_1', 'attr2_1', 'amb3_1', 'sinc1_1_f',
       'amb1_1', 'fun1_1_f', 'amb1_1_f', 'shar1_1', 'age_f'],
      dtype='object')

In [774]:
df_pair.iloc[:, 1:].columns[np.argsort(-my_clf.feature_importances_)[:11]]

Index(['int_corr', 'age', 'amb3_1_f', 'age_f', 'income_f', 'fun1_1', 'income',
       'field_cd', 'field_cd_f', 'sinc1_1_f', 'attr2_1'],
      dtype='object')

## Задание 5

In [656]:
grid = {
    'max_depth': range(1, 20, 2),
    'min_samples_leaf': range(3,10),
    'n_estimators': [1, 10, 25, 40],
    'criterion': ['gini', 'entropy'],
    'max_features': ['auto','sqrt','log2']
}
clf = RandomizedSearchCV(RandomForestClassifier(), param_distributions=grid)
clf.fit(X_train,y_train)
clf.best_params_

/home/scrinium/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


{'n_estimators': 40,
 'min_samples_leaf': 7,
 'max_features': 'log2',
 'max_depth': 17,
 'criterion': 'entropy'}

In [657]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=10, min_samples_leaf=5,
                            max_features='auto', max_depth=17)
rf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=17, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=5, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [658]:
f1_score(y_pred=rf.predict(X_test),y_true=y_test,average='macro')

0.49501611049348987